In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas
from math import sqrt
from brand_lists import *
from sklearn.model_selection import train_test_split

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', None)

In [10]:
#caricamento dei risultati della sentiment analysis per i tag (tabella USERS-TAGS VALUES)
user_tags_values = pd.read_csv('final_results/users_tags_values.csv')
user_list = np.unique(user_tags_values['User'].to_list())   

#SELEZIONE DEGLI UTENTI DA ANALIZZARE CON IL KNN (SCEGLIERE UNO DEI TRE)
#gold_standard = pd.read_excel('final_results/true_gold_standard.xlsx')       #SOLO UTENTI FOLLOWERS DELLE PAGINE UFFICIALI
gold_standard = pd.read_excel('final_results/sentiment_gold_standard.xlsx')    #SOLO UTENTI ETICHETTATI CON LA SENTIMENT ANALYSIS
#gold_standard = pd.read_excel('final_results/strong_gold_standard.xlsx')      #UTENTI INTERSEZIONE TRA FOLLOWERS E ETICHETTATI CON LA SENTIMENT ANALYSIS

#Raggruppamento dei due dataframe in dizionari per
#facilitare la ricerca nella creazione delle classi
dict_1 = dict(tuple(gold_standard.groupby('Brand')))  #raggruppa gli utenti del gold standard per Brand
dict_2 = dict(tuple(user_tags_values.groupby('User')))     #raggruppa gli utenti di USERS-TAGS table per VETTORI

In [11]:
#METODI PER LA CREAZIONE DELLE CLASSI


#Restituisce il vettore di un utente
def get_user_vector(user):
    user_values = dict_2.get(user)                     #riga dei valori di sentiment
    vector = []
    for tag in all_brands_tags:                     #per ogni tag
        vector.append(user_values[tag].values[0])   #copia il valore letto
    return np.array(vector)


#Restituisce la lista di utenti di una classe
def get_class_users(brand):
    brand_users_df = dict_1.get(brand)                 
    brand_users_list = np.unique(brand_users_df['User'].to_list())
    return brand_users_list


#Restituisce una lista di tuple-> (brand,User,[VETTORE] )
def get_class_followers_vectors(brand):
    vectors_list = []
    brand_users_list = get_class_users(brand)
    for user in brand_users_list:
        tern = (brand, user, get_user_vector(user)) #terna di valori ( brand,User,[VETTORE] )
        vectors_list.append(tern)
    return vectors_list
    
    
#riempimento degli utenti delle classi:  
all_classes = []
for brand in brands_dict.keys():    #per tutti i label
    all_classes.extend(get_class_followers_vectors(brand))  

In [12]:
#METODI PER IL K-N NEIGHBOURS

#Calcola e restituisce la distanza euclidea tra 2 vettori
def euclidean_distance(row1, row2):
    distance = 0.0
    for i in range(len(row1) - 1):
        distance += (row1[i] - row2[i])**2
    return sqrt(distance)


#Ordina le tuple di valori (Rank, Vettore) in modo crescente rispetto Rank
def sort_rank_list(tuple):
   return(sorted(tuple, key = lambda n: float(n[0]), reverse = False))


#Verifica se piu di K elementi hanno distanza uguale (restituisce True se si verifica)
def check_if_greater_than_k(score_and_label,k):
    treshold = score_and_label[0][0]
    boolean = True
    for a in range(k+1):
        if score_and_label[a][0]>treshold:
            boolean= False
            break
    return boolean

#Raccoglie tutti i primi label presenti in Rx DI PARI UGUAGLIANZA
def get_greater_k_labels(score_and_label):
    v=[]
    count = 0
    treshold = score_and_label[0][0]
    while(score_and_label[count][0]==treshold):
        v.append(score_and_label[count][1])
        count+=1
    return v

#Restituisce la lista dei primi elementi in Rx DI PARI UGUAGLIANZA
def get_greater_k_rank_elements(score_and_label):
  new=[]
  count = 0
  treshold = score_and_label[0][0]
  while(score_and_label[count][0]==treshold):
    new.append(score_and_label.pop(0))
  return new

#Calcola la distanza euclidea tra TUTTI gli elementi di C_test e C_training.
#Restituisce: [ Ux[1] , list[tuple:(distance,Uy[0])]   ]      
def get_rank_list(C_test, C_training):
    rank_list = []
    for Ux in C_test:                                       #LEGENDA: Ux[0]='brand' #Ux[1]='user' #Ux[2]=[VETTORE]
        Ux_rank = []
        for Uy in C_training:                               #LEGENDA: Uy[0]='brand' #Uy[1]='user' #Uy[2]=[VETTORE]
            distance = euclidean_distance(Ux[2], Uy[2])
            Uy_data = (distance,Uy[0],Uy[1])                #salva la tupla (distanza, brand (etichetta) di Uy)
            Ux_rank.append(Uy_data)                         #aggiunge la tupla alla lista di ranking di Ux
        tuple = (Ux[1], sort_rank_list(Ux_rank))            #salva la tupla (Username di Ux, lista di ranking di Ux)
        rank_list.append(tuple)
    return rank_list


#Controlla l'etichetta maggiormente presente e la restituisce
#In caso di parità di occorrenze restituisce una stringa vuota
def most_frequent_label(v):
    label_prediction = ""
    max_value = 0
    for label in np.unique(v):
        occurrence = v.count(label)
        if occurrence > max_value:
            max_value = occurrence
            label_prediction = label
        elif occurrence == max_value:
            label_prediction = ""
        else:
            continue
    return label_prediction


#Predizione della label
#Restituisce una lista di utenti con l'etichetta predetta
def get_k_predictions(rank_list, k):
    knn_result = []
    for result in rank_list:
        Ux = result[0]
        score_and_label = result[1]
        v = []
        #se piu dei primi K elementi hanno pari distanza:
        if check_if_greater_than_k(score_and_label,k):
            v = get_greater_k_labels(score_and_label)

        #altrimenti prendi solo i primi K:
        else:
            for i in range(k):
                v.append(score_and_label[i][1])

        label_predict = (Ux, most_frequent_label(v))
        if label_predict[1] != "":
            knn_result.append(label_predict)

    return knn_result
   
        
#Esegue tutti i metodi sopra definiti
def execute_knneighbour(C_train, C_test, k):
    rank_list = get_rank_list(C_test,C_train)
    knn = get_k_predictions(rank_list,k)
    print("[K = ",k,"]")
    print("total users in classes: ",len(all_classes))
    print("train users: ",len(C_train))
    print("test users: ",len(C_test))
    print("rank size: ",len(rank_list))
    print("k-NN predictions: ",len(knn))
    print('\n')
    return rank_list, knn

In [13]:
#DIVISIONE DEI DATI IN TRAINING E TEST
C_train, C_test  = train_test_split(all_classes,test_size=0.2)

print(len(all_classes))
print(len(C_test))
print(len(C_train))

331
67
264


In [14]:
#ESECUZIONE DEL KNN
rank_list_1, knn_1 = execute_knneighbour(C_train,C_test,1)
predicted_1 = pd.DataFrame(knn_1, columns=['User','predicted-label (K=1)']).sort_values(by=['User'])

rank_list_3, knn_3 = execute_knneighbour(C_train,C_test,3)
predicted_3 = pd.DataFrame(knn_3, columns=['User','predicted-label (K=3)']).sort_values(by=['User'])

rank_list_5, knn_5 = execute_knneighbour(C_train,C_test,5)
predicted_5 = pd.DataFrame(knn_5, columns=['User','predicted-label (K=5)']).sort_values(by=['User'])

rank_list_7, knn_7 = execute_knneighbour(C_train,C_test,7)
predicted_7 = pd.DataFrame(knn_7, columns=['User','predicted-label (K=7)']).sort_values(by=['User'])

[K =  1 ]
total users in classes:  331
train users:  264
test users:  67
rank size:  67
k-NN predictions:  25


[K =  3 ]
total users in classes:  331
train users:  264
test users:  67
rank size:  67
k-NN predictions:  23


[K =  5 ]
total users in classes:  331
train users:  264
test users:  67
rank size:  67
k-NN predictions:  31


[K =  7 ]
total users in classes:  331
train users:  264
test users:  67
rank size:  67
k-NN predictions:  37




In [23]:
t = predicted_7.reset_index(drop=True)
gold_standard.merge(t, on='User')

,User,Brand,predicted-label (K=7)
0,21Phylk,armani,armani
1,21Phylk,adidas,armani
2,AnteaterButters,lavazza,ferrero
3,AnteaterButters,barilla,ferrero
4,AnteaterButters,ferrero,ferrero
...,...,...,...
114,smithywannabe,amarelli,armani
115,tuscanysusie,alfaromeo,alfaromeo
116,tuscanysusie,barilla,alfaromeo
117,weetish,ferrero,ferrero
